# Land Cover Classification

In [1]:
import geoai

In [2]:
geoai.__version__

'0.7.1'

In [2]:
train_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/m_3807511_ne_18_060_20181104.tif"
train_landcover_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/m_3807511_ne_18_060_20181104_landcover.tif"
test_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/m_3807511_se_18_060_20181104.tif"

In [3]:
train_raster_path = geoai.download_file(train_raster_url)
train_landcover_path = geoai.download_file(train_landcover_url)
test_raster_path = geoai.download_file(test_raster_url)

File already exists: m_3807511_ne_18_060_20181104.tif
File already exists: m_3807511_ne_18_060_20181104_landcover.tif
File already exists: m_3807511_se_18_060_20181104.tif


# Visualize Sample data

In [4]:
geoai.view_raster(train_landcover_path,basemap = train_raster_path)

In [7]:
geoai.view_raster(test_raster_path)

In [5]:
geoai.view_raster('Crop_Classified_for_Unet.tif')

In [6]:
out_folder = 'landcover'
tiles = geoai.export_geotiff_tiles(
    in_raster = train_raster_path,
    out_folder= out_folder,
    in_class_data= train_landcover_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

Detected in_class_data as raster: m_3807511_ne_18_060_20181104_landcover.tif
Raster CRS: EPSG:26918
Raster dimensions: 10049 x 12530

Raster info for m_3807511_ne_18_060_20181104.tif:
  CRS: EPSG:26918
  Dimensions: 10050 x 12531
  Resolution: (0.6, 0.6)
  Bands: 4
  Bounds: BoundingBox(left=440055.0, bottom=4295910.600000001, right=446085.0, top=4303429.2)
Found 12 unique classes in raster: [ 1  2  3  4  5  6  7  8  9 10 11 12]


Generated: 1872, With features: 1872: 100%|██████████| 1872/1872 [01:18<00:00, 23.89it/s]



------- Export Summary -------
Total tiles exported: 1872
Tiles with features: 1872 (100.0%)
Average feature pixels per tile: 524244.9
Output saved to: landcover

------- Georeference Verification -------


In [5]:
# Train U-Net model
geoai.train_segmentation_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/unet_models",
    architecture="unet",
    encoder_name="resnet34",
    encoder_weights="imagenet",
    num_channels=4,
    num_classes=13,
    batch_size=8,
    num_epochs=50,
    learning_rate=0.001,
    val_split=0.2,
    verbose=True,
    plot_curves=True,
)

Using device: cuda
Found 1872 image files and 1872 label files
Training on 1497 images, validating on 375 images
Starting training with unet + resnet34
Model parameters: 24,441,245
Epoch: 0, Batch: 0/188, Loss: 2.6801, Time: 35.08s
Epoch: 0, Batch: 10/188, Loss: 1.1764, Time: 12.17s
Epoch: 0, Batch: 20/188, Loss: 0.5206, Time: 13.45s
Epoch: 0, Batch: 30/188, Loss: 0.6264, Time: 12.23s
Epoch: 0, Batch: 40/188, Loss: 0.4204, Time: 12.11s
Epoch: 0, Batch: 50/188, Loss: 0.2154, Time: 12.13s
Epoch: 0, Batch: 60/188, Loss: 0.3521, Time: 12.21s
Epoch: 0, Batch: 70/188, Loss: 0.2739, Time: 12.32s
Epoch: 0, Batch: 80/188, Loss: 0.3515, Time: 12.77s
Epoch: 0, Batch: 90/188, Loss: 0.3997, Time: 12.24s
Epoch: 0, Batch: 100/188, Loss: 0.4023, Time: 12.62s
Epoch: 0, Batch: 110/188, Loss: 0.2782, Time: 12.41s
Epoch: 0, Batch: 120/188, Loss: 0.3226, Time: 12.35s
Epoch: 0, Batch: 130/188, Loss: 0.2009, Time: 12.32s
Epoch: 0, Batch: 140/188, Loss: 0.2421, Time: 12.29s
Epoch: 0, Batch: 150/188, Loss: 0.3

# Run Inference
Now we will use the trained model to make predictions on the test image.

In [7]:
#Define paths
masks_path = "naip_test_semantic_prediction.tif"
model_path = "landcover/unet_models/best_model.pth"

In [8]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))


CUDA available: True
GPU name: Quadro P2200


In [9]:
# Run Semantic Segmentation inference
geoai.semantic_segmentation(
    input_path = test_raster_path,
    output_path= masks_path,
    model_path=model_path,
    architecture="unet",
    encoder_name="resnet34",
    num_channels=4,
    num_classes=13,
    window_size=512,
    overlap=256,
    batch_size=4,
    device="cuda" #if geoai.is_cuda_available() else "cpu",
)

Processing 1911 windows...


2000it [01:36, 20.75it/s]                          


Predicted classes: 9 classes, Background: 0.0%
Inference completed in 135.25 seconds
Saved prediction to naip_test_semantic_prediction.tif


In [10]:
geoai.write_colormap(masks_path, train_landcover_path, output=masks_path)

In [11]:
geoai.view_raster(masks_path, basemap=test_raster_url)